In [1]:
import polars as pl
import numpy as np
from sklearn.neighbors import KernelDensity
from scipy.optimize import minimize
from scipy.special import gammaln
from utils import donation_data, rfm_data, p1x_data

import altair as alt
from IPython.display import display_markdown

In [2]:
data = donation_data()
rfm_df = rfm_data(data)
rfm_summary = p1x_data(rfm_df)
rfm_summary.collect()

P1X,t_x,np1x,Count
i8,i32,i32,u32
6,6,6,1203
5,6,6,728
4,6,6,512
3,6,6,357
2,6,6,234
…,…,…,…
1,3,6,129
2,2,6,613
1,2,6,277


In [3]:
# Possible recency/frequency patterns 
n = 6
n * (n + 1)/2 + 1

22.0

### Parameter Estimation

In [4]:
# def bgbb_est(rfm_data, guess={'alpha': 1, 'beta': 0.5, 'gamma': 0.5, 'delta': 2.5}):
    
#     def log_likelihood(param):
        

rfm_array = rfm_summary.collect().to_numpy()
p1x, t_x, n, num_donors = [*rfm_array.T]

# Sample parameters
alpha = 1.20352083040498
beta = 0.749714243061896
gamma = 0.656712169147878
delta = 2.78340801635898

B_alpha_beta = np.exp(gammaln(alpha)+gammaln(beta)-gammaln(alpha+beta))
B_gamma_delta = np.exp(gammaln(gamma)+gammaln(delta)-gammaln(gamma+delta))

n_t_x_1 = n - t_x - 1

L_x_t_x_n = np.zeros((n.shape[0], n[0]+1))
L_x_t_x_n[:,0] = np.exp(gammaln(alpha+p1x)+gammaln(beta+n-p1x)-gammaln(alpha+beta+n))/B_alpha_beta*np.exp(gammaln(gamma)+gammaln(delta+n)-gammaln(gamma+delta+n))/B_gamma_delta
L_x_t_x_n

array([[0.11285289, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.01363861, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.00458606, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.00299995, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.00351144, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.00756894, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.01363861, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.00458606, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.00299995, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.00351144, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0. 